In [1]:
import tensorflow as tf
import keras
from keras.layers import LeakyReLU, BatchNormalization, Input, Activation, Concatenate
import numpy as np
from numpy import ones
from numpy import zeros
from keras.initializers import RandomNormal
from numpy.random import randint
from matplotlib import pyplot

In [2]:
imgs_shape = (32,32,3)

In [3]:
def discriminator():
  init = RandomNormal(stddev=0.02)
  in_src_image = Input(shape=imgs_shape)
  in_target_image = Input(shape=imgs_shape)
  merged = Concatenate()([in_src_image, in_target_image])

  d = tf.keras.layers.Conv2D(32, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
  d = LeakyReLU(alpha=0.2)(d)
  
  d = tf.keras.layers.Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)

  d = tf.keras.layers.Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)

  d = tf.keras.layers.Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)

  d = tf.keras.layers.Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
  patch_out = Activation('sigmoid')(d)
  
  model = keras.models.Model([in_src_image, in_target_image], patch_out)

  opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
  return model

In [4]:
def generator():
    X = Input(shape = imgs_shape)
  
    conv1 = tf.keras.layers.Conv2D( 32 , kernel_size=( 5 , 5 ) , strides=1 )( X )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )
    conv1 = tf.keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1)( conv1 )
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )

    conv2 = tf.keras.layers.Conv2D( 64 , kernel_size=( 5 , 5 ) , strides=1)( conv1 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )
    conv2 = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 )( conv2 )
    conv2 = tf.keras.layers.LeakyReLU()( conv2 )

    conv3 = tf.keras.layers.Conv2D( 128 , kernel_size=( 5 , 5 ) , strides=1 )( conv2 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )
    conv3 = tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 )( conv3 )
    conv3 = tf.keras.layers.LeakyReLU()( conv3 )

    bottleneck = tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='tanh' , padding='same' )( conv3 )

    concat_1 = tf.keras.layers.Concatenate()( [ bottleneck , conv3 ] )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_1 )
    conv_up_3 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' )( conv_up_3 )

    concat_2 = tf.keras.layers.Concatenate()( [ conv_up_3 , conv2 ] )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_2 )
    conv_up_2 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' )( conv_up_2 )

    concat_3 = tf.keras.layers.Concatenate()( [ conv_up_2 , conv1 ] )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( concat_3 )
    conv_up_1 = tf.keras.layers.Conv2DTranspose( 3 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu')( conv_up_1 )
  
    model = tf.keras.models.Model( X , conv_up_1 )
    return model


In [5]:
def load_dataset(filename):
  data = np.load(filename)
  X1, X2 = data["arr_0"], data["arr_1"]
  X1 = (X1 - 127.5)/127.5
  X2 = (X2 - 127.5)/127.5
  return [X1,X2]

In [6]:
def generate_real_images(dataset, n, patch_shape):
  trainA, trainB = dataset
  ix = randint(0, trainA.shape[0],n)
  X1, X2 = trainA[ix], trainB[ix]
  y = ones((n, patch_shape, patch_shape,1))
  return [X1,X2], y

In [7]:
def generate_fake_images(model, sample, patch_shape):
  X = model.predict(sample)
  y = zeros((len(X), patch_shape, patch_shape, 1))
  return X,y

In [ ]:
dataset = load_dataset('dataset\cifar10.npz')
print("Loaded", dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]
disc_model = discriminator()
gen_model = generator() 

In [9]:
gen_model.load_weights("..\Weights\model_060.h5")

In [ ]:
n = 3
#dataset = load_dataset('cifar10.npz')

[realA, realB], _ = generate_real_images(dataset, n, 1)
fakeB, _ = generate_fake_images(gen_model, realA, 1)
realA = (realA+1)/2.0
realB = (realB+1)/2.0
fakeB = (fakeB+1)/2.0

# plot real source images
for i in range(n):
  pyplot.subplot(3, n, 1+i)
  pyplot.axis("off")
  pyplot.imshow(realA[i])
  
#plot generated target images  
for i in range(n):
  pyplot.subplot(3, n, 1+n+i)
  pyplot.axis("off")
  pyplot.imshow(fakeB[i])

# plot real target images
for i in range(n):
  pyplot.subplot(3, n, 1+n*2+i)
  pyplot.axis("off")
  pyplot.imshow(realB[i])